In [55]:
!rm -rf /kaggle/working/multimodal-eq-sizing
!git clone https://github.com/brianrp09232000/multimodal-eq-sizing.git /kaggle/working/multimodal-eq-sizing
!pip install -r /kaggle/working/multimodal-eq-sizing/requirements.txt

Cloning into '/kaggle/working/multimodal-eq-sizing'...
remote: Enumerating objects: 167, done.
remote: Counting objects: 100% (167/167), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 167 (delta 74), reused 16 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (167/167), 39.09 KiB | 3.26 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [56]:
import sys
import pathlib
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [57]:
np.seterr(invalid="ignore")

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

In [58]:
repo_root = pathlib.Path("/kaggle/working/multimodal-eq-sizing")
sys.path.append(str(repo_root))

In [59]:
from src.data.loaders import get_return_data, get_excess_return, get_vix_data, get_spread_z

In [60]:
def get_date_range(df: pd.DataFrame) -> tuple:
    grouped_by_date = df.groupby(["ticker"]).agg(['min', 'max', 'count'])["Date"]
    start = grouped_by_date["min"].min()
    end = grouped_by_date["max"].max()
    return start, end

In [ ]:
df = get_return_data("/kaggle/input/news-trading/return_data.csv")
start, end = get_date_range(df_ticker_return)

# Compute excess return

In [ ]:
def add_excess_return(df, start, end):
    excess_return_df = get_excess_return(df_ticker_return, start, end)
    df = df.merge(excess_return_df, on=["ticker", "Date"], how="left")
    return df

In [ ]:
df = add_excess_return(df)

# Add market regime VIX z-score

In [ ]:
def add_vix_z(df, start, end):
    vix_z_df = get_vix_data(start, end)
    df = df.merge(excess_return_df, on=["Date"], how="left")
    return df

In [ ]:
df = add_vix_z(df)

# Add spread z-score

In [64]:
def add_spread_z(existing_df: pd.DataFrame) -> pd.DataFrame:
    """
    Use existing OHLCV df, pull buffered history, compute young-safe spread_z on the combined
    Then merge back only the target window rows to prevent nulls.
    """
    df = existing_df.copy()
    start, end = df["Date"].min(), df["Date"].max()

    tickers = sorted(df[ticker_col].unique())
    fetch_start = start - timedelta(days=buffer_days)
    fetch_end   = end

    # You already have get_tickers_history(tickers, start, end)
    hist = get_tickers_history(tickers, fetch_start, fetch_end)
    hist["Date"] = pd.to_datetime(hist["Date"], utc=True)

    # Combine buffer + existing; keep existing rows on overlap
    combined = pd.concat([hist, df], ignore_index=True)
    combined = combined.sort_values([ticker_col, "Date"])
    combined = combined.drop_duplicates(subset=[ticker_col, "Date"], keep="last")

    # Compute young-safe spread_z on the full combined range
    combined = get_spread_z(combined)

    # Merge only computed columns back to target window
    cols_to_merge = [ticker_col, date_col, "spread_z"]
    out = df.merge(combined[cols_to_merge], on=[ticker_col, date_col], how="left")

    # Final minimal, causal clean-up to guarantee NON-NULL spread_z in target window:
    # 1) per-ticker forward-fill (past only), 2) same-day cross-section median, 3) final 0
    out["spread_z"] = (
        out.groupby(ticker_col)["spread_z"].ffill()
           .fillna(out.groupby(date_col)["spread_z"].transform("median"))
           .fillna(0.0)
    ).clip(-3, 3)

    return out

In [65]:
df = add_spread_z(df)

# Final Complete Dataset

In [ ]:
df

In [70]:
!rm -rf /kaggle/working/multimodal-eq-sizing